In [52]:
import pandas as pd
import os
import numpy as np 
import glob
import csv
import cassandra

In [53]:
#

In [54]:

filepath = os.getcwd() + '/event_data'

for root, dirs , files in os.walk(filepath):
    
    required_paths = glob.glob(os.path.join(root,'*'))

In [55]:
# full data rows

full_data_row = []
for path in required_paths:
    with open(path,'r') as _:
        csvrows = csv.reader(_)
        next(csvrows)
        for line in csvrows:
            full_data_row.append(line)    

In [56]:
len(full_data_row )

8056

In [110]:
#

In [57]:
dialect = csv.register_dialect('mydialect',quoting = csv.QUOTE_ALL,skipinitialspace=True)

with open('full_data.csv',encoding='utf8',mode='w',newline='') as reader:
    csv_writer = csv.writer(reader,dialect=dialect)

    csv_writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])

    for row in full_data_row:

        if row[0] == '':
            continue
        csv_writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [58]:
with open('full_data.csv',encoding='utf8',mode='r',newline='') as r :
    print(sum(1 for l in r))

6821


In [7]:
from cassandra.cluster import Cluster

In [26]:
from cassandra.auth import PlainTextAuthProvider
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')

try:
    conn = Cluster(auth_provider = auth_provider)
    session = conn.connect()
except Exception as e:
    print(e)

In [46]:
create_keyspace = "create keyspace if not exists sparkly \n "
create_keyspace += " \t with replication = {'class':'SimpleStrategy', 'replication_factor' : 1} "



try:
    session.execute(create_keyspace)
    session.set_keyspace('sparkly')
except Exception as e:
    print(e)


In [47]:
# Create table to store the values


create_session_item ="""CREATE TABLE IF NOT EXISTS session_item (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"""


try:
    session.execute(create_session_item)
    print('Table Created')
except Exception as e:
    print(f'Table Creation failed error : {e}')

Table Created


In [60]:
# Insert data
file_n = 'full_data.csv'
with open(file_n,encoding='utf8',newline='',mode='r') as f:
    read_f = csv.reader(f)
    next(read_f)

    for line in read_f:
        query = " insert into session_item (artist, song, length, sessionId, itemInSession) \n "
        query += " values (%s, %s, %s, %s, %s)"
        session.execute(query,(line[0], line[10], line[5], int(line[8]), int(line[3])))



In [61]:
# Query 2

In [62]:
# Creating table for query2 
create_query2 = """CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """

try: 
    session.execute(create_query2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [64]:
with open(file_n, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )

In [67]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")



In [69]:
conn.shutdown()